#<h2>**1 Importing various libraries for the data analysis:**</h2>

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import scipy
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import gc
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import shutil
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
#from py7zr import unpack_7zarchive
import math
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix, hstack
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from prettytable import PrettyTable
from sklearn.linear_model import RidgeCV
import pickle
import zipfile
from tqdm import tqdm
import os
from sklearn.linear_model import Ridge
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Input, Embedding, GRU, Dense,Flatten
from tensorflow.keras.models import Model,load_model
from numpy import zeros
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from contextlib import contextmanager
tf.keras.backend.clear_session()

#<h2>**2 Utility Functions:**</h2>

In [ ]:
def split_categories(category):
    '''
    function that splits the category column in the dataset and creates 3 new columns:
    'main_category','sub_cat_1','sub_cat_2'
    '''
    try:
        sub_cat_1,sub_cat_2,sub_cat_3 = category.split("/")
        return sub_cat_1,sub_cat_2,sub_cat_3
    except:
        return ("No label","No label","No label")

def create_split_categories(data):
    '''
    function that creates 3 new columns using split_categories function
    : 'main_category','sub_cat_1','sub_cat_2'
    '''
    data['main_category'],data['sub_cat_1'],data['sub_cat_2']=zip(*data['category_name'].\
                                                                  apply(lambda x: split_categories(x)))

def log_price(price):
    return np.log1p(price)#changes

#https://www.kaggle.com/valkling/mercari-rnn-2ridge-models-with-notes-0-42755
def countwords(text):
    try:
        if text == 'No description yet':
            return 0
        else:
            text = text.lower()
            words = [w for w in text.split(" ")]
            return len(words)
    except: 
        return 0

def fill_nan(dataset):
    '''
    Function to fill the NaN values in various columns
    '''
    dataset["item_description"].fillna("No description yet",inplace=True)
    dataset["brand_name"].fillna("missing",inplace=True)
    dataset["category_name"].fillna("missing",inplace=True)

def rank_category(dataset,column_name):
    '''This function takes a column name which is categorical and returns the categories with rank'''
    counter = dataset[column_name].value_counts().index.values
    total = list(dataset[column_name])
    ranked_cat = {}
    for i in range(1,len(counter)+1):
        ranked_cat.update({counter[i-1] : i})
    return ranked_cat,len(counter)

def encode_ranked_category(train,test,column):
    '''
    This function calls the rank_category function and returns the encoded category column    '''
    train[column] = train[column].astype('category')
    test[column] = test[column].astype('category')
    
    cat_list = list(train[column].unique())
    ranked_cat_tr,count = rank_category(train,column)

    encoded_col_tr = []
    encoded_col_te = []

    for category in train[column]:
        encoded_col_tr.append(ranked_cat_tr[category])

    for category in test[column]:
        if category in cat_list:
            encoded_col_te.append(ranked_cat_tr[category])
        else:
            encoded_col_te.append(0)
    
    encoded_col_tr = np.asarray(encoded_col_tr)
    encoded_col_te = np.asarray(encoded_col_te)
    return encoded_col_tr,encoded_col_te,count

def tokenize_text(train,test,column):
    global t
    t = Tokenizer()
    t.fit_on_texts(train[column].str.lower())
    vocab_size = len(t.word_index) + 1
    # integer encode the documents
    encoded_text_tr = t.texts_to_sequences(train[column].str.lower())
    encoded_text_te = t.texts_to_sequences(test[column].str.lower())
    return encoded_text_tr,encoded_text_te,vocab_size

def data_gru(train,test):
    
    global max_length,desc_size,name_size
    encoded_brand_tr,encoded_brand_te,brand_len = encode_ranked_category(train,test,'brand_name')
    encoded_main_cat_tr,encoded_main_cat_te,main_cat_len = encode_ranked_category(train,test,'main_category')
    encoded_sub_cat_1_tr,encoded_sub_cat_1_te,sub_cat1_len = encode_ranked_category(train,test,'sub_cat_1')
    encoded_sub_cat_2_tr,encoded_sub_cat_2_te,sub_cat2_len = encode_ranked_category(train,test,'sub_cat_2')
    tokenized_desc_tr,tokenized_desc_te,desc_size = tokenize_text(train,test,'item_description')
    tokenized_name_tr,tokenized_name_te,name_size = tokenize_text(train,test,'name')
      
    max_length = 160
    desc_te_padded = pad_sequences(tokenized_desc_te, maxlen=max_length, padding='post')
    del tokenized_desc_tr,tokenized_desc_te

    name_te_padded = pad_sequences(tokenized_name_te, maxlen=10, padding='post')
    del tokenized_name_tr,tokenized_name_te

    gc.collect()

    test_inputs = [name_te_padded,desc_te_padded,encoded_brand_te.reshape(-1,1),\
                    encoded_main_cat_te.reshape(-1,1),encoded_sub_cat_1_te.reshape(-1,1),\
                    encoded_sub_cat_2_te.reshape(-1,1),test['shipping'],\
                    test['item_condition_id'],test['wc_desc'],\
                    test['wc_name']] 
    return test_inputs

def rmsle_compute(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    score = np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2)))
    return score

def scale_back(x):
    '''
    Function to inverse transform the scaled values
    '''
    x= np.expm1(y_scalar.inverse_transform(x.reshape(-1,1))[:,0])#changes
    return x

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#<h2>**3 Function 1:**</h2>

In [ ]:
def final_fun_1(x):
    '''This function takes in the raw input, preprocess it, trains and returns the final predictions'''
    ############################
    #step1: load the train file
    ############################
    gc.collect()
    train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/self_case_study_projects/project_1/train.tsv',sep='\t')
    test = x
    print("Finished loading the files....\n")
    print("train: {0}\ntest: {1}\n".format(train.shape,test.shape))
    ##########################################################
    #step2: Data cleaning and preprocessing of train and test
    ##########################################################
    #https://www.kaggle.com/valkling/mercari-rnn-2ridge-models-with-notes-0-42755
    train = train.drop(train[(train.price < 3.0)].index)
    fill_nan(train)
    fill_nan(test)
    print("filled nan\n")

    train['wc_desc'] = train['item_description'].apply(lambda x: countwords(x))
    test['wc_desc'] = test['item_description'].apply(lambda x: countwords(x))
    train['wc_name'] = train['name'].apply(lambda x: countwords(x))
    test['wc_name'] = test['name'].apply(lambda x: countwords(x))
    create_split_categories(train)
    create_split_categories(test)
    print("Completed data cleaning and preprocessing\n")

    # train test split
    #X_train, X_test= train_test_split(train, train_size=0.99, random_state=123)
    print("shape of train: {}".format(train.shape))
    print("shape of test: {}".format(test.shape))

    global y_scalar,DESC_LEN,NAME_LEN
    y_scalar = StandardScaler()#changes
    y_train = y_scalar.fit_transform(log_price(train['price']).values.reshape(-1, 1))#changes

    DESC_LEN = train.wc_desc.max() + 1
    NAME_LEN = train.wc_name.max() + 1
    #################################
    #step:3 Featurizing
    #################################
    test_inputs = data_gru(train,test)
    #################################
    #step:4 compiling and predicting
    #################################
    model_1 = load_model('/content/gdrive/My Drive/Colab Notebooks/self_case_study_projects/project_1/GRU_model/final/GRU_lr-003-0.328.hdf5')
    te_preds_m1 = model_1.predict(test_inputs,batch_size = 2**10,verbose = 1)
    model_2 = load_model('/content/gdrive/My Drive/Colab Notebooks/self_case_study_projects/project_1/GRU_model/final/GRU_lr-003-0.326.hdf5')
    te_preds_m2 = model_1.predict(test_inputs,batch_size = 2**10,verbose = 1)
    del test_inputs
    gc.collect()
    #https://machinelearningmastery.com/model-averaging-ensemble-for-deep-learning-neural-networks/

    y_hats = np.array([te_preds_m1,te_preds_m2]) #making an array out of all the predictions
    # mean across ensembles
    mean_preds = np.mean(y_hats, axis=0)
    return scale_back(mean_preds)

#<h3>**Price predicitons from final_func_1 with one item**</h3>
Giving a single test data as input for price prediction

In [ ]:
#call for function_1
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/self_case_study_projects/project_1/test.tsv',sep='\t')
predictions = final_fun_1(test[4:5])
#https://www.geeksforgeeks.org/python-convert-two-lists-into-a-dictionary/
result = {test["name"][i]: predictions[i] for i in range(len(predictions))}
print("\n##############################################\n")
print("\nThe price predicted for the given input(s): \n")
print("\nItem(s)\tpredicted_price\n")
for k,v in result.items():
    print("{0}\t{1}".format(k,v))

Finished loading the files....

train: (1482535, 8)
test: (1, 7)

filled nan

Completed data cleaning and preprocessing

shape of train: (1481661, 13)
shape of test: (1, 12)
1/1 [==============================] - 0s 1ms/step

##############################################


The price predicted for the given input(s): 


Item(s)	predicted_price

Breast cancer "I fight like a girl" ring	8.730072021484375


#<h3>**Price predicitons from final_func_1 with more items**</h3>

In [ ]:
predictions = final_fun_1(test[:10])
#https://www.geeksforgeeks.org/python-convert-two-lists-into-a-dictionary/
results = {test["name"][i]: predictions[i] for i in range(len(predictions))}
print("\n##############################################\n")
print("\nThe price predicted for the given input(s): \n")
print("\nItem(s)\tpredicted_price\n")
for k,v in results.items():
    print("{0}\t{1}".format(k,v))

Finished loading the files....

train: (1482535, 8)
test: (10, 7)

filled nan

Completed data cleaning and preprocessing

shape of train: (1481661, 13)
shape of test: (10, 12)
1/1 [==============================] - 0s 1ms/step

##############################################


The price predicted for the given input(s): 


Item(s)	predicted_price

Breast cancer "I fight like a girl" ring	6.8716912269592285
25 pcs NEW 7.5"x12" Kraft Bubble Mailers	11.653888702392578
Coach bag	42.083763122558594
Floral Kimono	12.492950439453125
Life after Death	8.730072021484375
iPhone 6 Plus or 6s Plus Vodka pink case	8.598246574401855
Vintage Cameo Pendant & Brooch Pin	10.062675476074219
Rose Gold Stainless Steel Quartz Watch	31.60715103149414
Daisy Marc Jacobs 3.4oz	45.37272262573242
Rose Brushes and Silicone Sponge	10.792123794555664


#<h2>**4 Function 2:**</h2>

In [ ]:
def final_fun_2(X,Y):
    '''This function takes in the raw input and its true target values and returns the metric on true and predicted'''
    score = rmsle_compute(final_fun_1(X),Y)
    return score

In [ ]:
#call for function 2
test1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/self_case_study_projects/project_1/test.tsv',sep='\t')
rmsle_score = final_fun_2(test1[4:5],[13])

Finished loading the files....

train: (1482535, 8)
test: (1, 7)

filled nan

Completed data cleaning and preprocessing

shape of train: (1481661, 13)
shape of test: (1, 12)
1/1 [==============================] - 0s 1ms/step


#<h3>**RMSLE score from function 2**</h3>

In [ ]:
print("RMSLE score(s) for the given test data:{}".format(rmsle_score))

RMSLE score(s) for the given test data:0.36383598180642096
